In [ ]:
import pandas as pd
import ast

In [ ]:
# L'objectiu és, donada una notícia, retornar les tags que més encaixen per a aquesta
# En primer lloc recuperarem el dataset sencer, ja que ho provarem amb totes les etiquetes

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('final_dataset_v1.csv', delimiter=',', encoding='utf-8', low_memory=False)


In [3]:
len(df)

147093

In [3]:
############################################################################### LET'S START WITH THE NEWS

In [18]:
df_with_tags = df[df['tags'].notnull()]

In [14]:
# Llibreries necessàries
import os
import openai
import dotenv
from openai import OpenAI
import sys
import pandas as pd
import json
from dotenv import load_dotenv
import re

# Carreguem la API KEY
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

# Models disponibles
models = openai.models.list()
model_list = [model.id for model in models.data]
print(model_list)

['gpt-4o-audio-preview-2024-10-01', 'gpt-4o-mini-audio-preview', 'chatgpt-4o-latest', 'gpt-4o-mini-audio-preview-2024-12-17', 'o1-2024-12-17', 'gpt-4o-2024-11-20', 'gpt-4o-mini-realtime-preview', 'dall-e-2', 'o1', 'gpt-3.5-turbo', 'o1-preview-2024-09-12', 'gpt-3.5-turbo-0125', 'o1-preview', 'gpt-3.5-turbo-instruct', 'babbage-002', 'o1-mini-2024-09-12', 'whisper-1', 'dall-e-3', 'gpt-4-1106-preview', 'omni-moderation-latest', 'omni-moderation-2024-09-26', 'tts-1-hd-1106', 'gpt-4', 'gpt-4o-2024-05-13', 'tts-1-hd', 'davinci-002', 'gpt-4o', 'gpt-4o-2024-08-06', 'text-embedding-ada-002', 'gpt-4-0613', 'gpt-4-turbo', 'tts-1', 'tts-1-1106', 'gpt-3.5-turbo-instruct-0914', 'gpt-4-turbo-preview', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-4o-audio-preview', 'text-embedding-3-small', 'gpt-4-turbo-2024-04-09', 'gpt-3.5-turbo-1106', 'gpt-4o-realtime-preview-2024-10-01', 'gpt-3.5-turbo-16k', 'gpt-4o-audio-preview-2024-12-17', 'text-embedding-3-large', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-4

In [6]:
# Estudiarem com de positives o negatives són les notícies
template = '''
The following is a news article. Read it and perform the task that follows. Respond with a list as it is explained later.


####################


{article}


####################

The following is a list of pairs "id", "desc" for all possible tags. 


####################


{tags}


####################


Task: Determine the the tags that best fit the article given a list of existing tags.


1. Instruction: Provide a list of tags that best fit the article. The format shoud be a list of dictionaries, where each dictionary has three keys: "id", "desc" and "main". The "id" key should contain the id of the tag, and the "desc" key should contain the description of the tag and main should be True only for the main tag for the article.

2. Tags provided: you are provided with a json file with pairs of "id" and "desc" tags, the "desc" are the descriptions of the tags. You will have to choose the tags that best fit the article from this list. Do not add more than 5 tags, it is not necessary to put 5 tags if you think that less tags are enough. Do not return anything except the list of tags that best fit the article. Do not include the article in your response nor anything else.

'''

In [15]:
# Estudiarem com de positives o negatives són les notícies
template_cat = '''
A continuació es presenta un article de notícies. Llegeix i realitza la tasca que segueix. Respon amb una llista tal com s'explica més endavant. Ets un assistent periodístic que ajudarà a escollir les etiquetes ideals per una notícia.

####################


{article}


####################

A continuació es presenta una llista de parelles "id", "desc" per a tots els tags possibles.

####################


{tags}


####################


Tasca: Proposa les etiquetes que millor s'adapten a l'article donat una llista d'etiquetes existents.

1. Instrucció: Proporcioneu una llista d'etiquetes que millor s'adaptin a l'article. El format ha de ser una llista de diccionaris, on cada diccionari té tres claus: "id", "desc" i "main". La clau "id" ha de contenir l'identificador de l'etiqueta, la clau "desc" ha de contenir la descripció de l'etiqueta, i "main" ha de ser True només per a l'etiqueta principal de l'article. Escollir les etiquetes que millor s'adapten a l'article d'aquesta llista. Suggerir entre 5 i 15 etiquetes perquè el periodista pugui elegir. Hi ha moltes etiquetes de països, pensa a afegir alguna sobre un passa la notícia. No tornis res més que la llista d'etiquetes que millor s'adapten a l'article. No incloguis l'article en la teva resposta ni res més.

'''

In [16]:
client = OpenAI()

def generate_prompt(article, templ):
    article_cos = article['cos']
    article_cos = str(article_cos)
    prompt = templ.replace("{article}", article_cos)
    prompt = prompt.replace("{tags}", open("tags.json", "r").read())
    return prompt

def generate_api_response_model_4o(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content, completion

In [19]:
# Let's get a random row from df_with_tags
row = df_with_tags.sample()

row

,id,data,entradeta,permatitle,avantitol,tematiques,tags,titol,titol_portada,comarques,cos,redactor,imatges
143189,3316897,2024-10-17,Sinwar havia assumit des de principis d'agost ...,Israel investiga si ha matat el líder de Hamas...,NaN,"[{'id': 'PUACT_MON', 'desc': 'Món', 'main': Tr...","[{'id': 'ISRAEL', 'desc': 'Israel', 'main': Fa...","Israel mata a Gaza Yahya Sinwar, líder de Hama...",NaN,NaN,"El líder de Hamas, Yahya Sinwar, va morir aque...",Redacció/Agències,[{'text': 'https://img.3cat.cat/multimedia/jpg...


In [20]:
print(row['entradeta'].iloc[0])

Sinwar havia assumit des de principis d'agost el lideratge polític del grup islamista, després de l'assassinat d'Ismail Haniyeh a l'Iran


In [21]:
prompt = generate_prompt(row, template_cat)
response = generate_api_response_model_4o(prompt)

In [21]:
print(row.iloc[0]['cos'])

Viure en una casa de palla és tendència. Fa més d'una dècada, la majoria d'arquitectes pensaven que fer cases amb murs de palla era un disbarat. Anys després, és un material que va a l'alça en la construcció de cases ecològiques. No és una opció més econòmica, però sí més sostenible i més eficient energèticament. La temperatura es manté estable a 19 graus. 

La finestra de la veritat destaca sobre l'arrebossat d'argila (Nati Adell / Catalunya Ràdio)

A Mirco Zeccheto, d'origen italià, se'l coneix com l'arquitecte de la palla.

"No passes fred a l'hivern, ni calor a l'estiu. La humitat és la que toca. Els materials naturals, com la terra, el suro o el cànem l'autoregulen. Si n'hi ha massa, la treu, i si en té poca, l'agafa."


L'arquitecte, el propietari i la directora de Firhàbitat, a la casa de palla d'OIvan (Nati Adell / Catalunya Ràdio)

Més benestar 
El més valorat en aquest tipus de cases és el confort i el benestar que aporta estar envoltat de materials vius, tal com explica Zech

In [22]:
print(row.iloc[0]['tags'])

[{'id': 'ISRAEL', 'desc': 'Israel', 'main': False}, {'id': 'GAZA', 'desc': 'Gaza', 'main': False}, {'id': 'HAMASISRAEL', 'desc': 'Guerra a Gaza', 'main': False}, {'id': 'HAMAS', 'desc': 'Hamas', 'main': True}]


In [23]:
print(response[0])

```json
[
    {"id":"HAMAS", "desc":"Hamas", "main":true},
    {"id":"ISRAEL", "desc":"Israel", "main":false},
    {"id":"PALESTINA", "desc":"Palestina", "main":false},
    {"id":"PROXIMORIENT", "desc":"Pròxim Orient", "main":false},
    {"id":"GAZA", "desc":"Gaza", "main":false},
    {"id":"TERRORISME", "desc":"Terrorisme", "main":false},
    {"id":"JIHADISME", "desc":"Jihadisme", "main":false},
    {"id":"INTERNACIONA", "desc":"Internacional", "main":false},
    {"id":"POLITICA", "desc":"Política", "main":false},
    {"id":"MEDICLIMA", "desc":"Medi ambient i crisi climàtica", "main":false},
    {"id":"SOCIETAT", "desc":"Societat", "main":false},
    {"id":"ONU", "desc":"ONU", "main":false},
    {"id":"DRETSHUMANS", "desc":"Drets humans", "main":false},
    {"id":"GUERRACIVIL", "desc":"Guerra Civil espanyola", "main":false}
]
```


In [60]:
print(response[1].usage)

CompletionUsage(completion_tokens=119, prompt_tokens=11754, total_tokens=11873, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
